In [ ]:
my_path = "my path"

In [ ]:
%cd my_path

## Packages

In [ ]:
#pytangle
import pytangle
from pytangle.api import API
import datetime
from datetime import date
from datetime import datetime

token = "my token"

#selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#login daten instagram
with open("insta_credentials.txt", "rt") as f:
    login_data = f.read().split()

#ct
import pandas as pd
import requests
from IPython.display import clear_output 
import time
from fake_useragent import UserAgent
ua = UserAgent()
import re
import json
import os
import csv

#email for error message
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import smtplib
import logging

## Logfile

In [ ]:
logging.basicConfig(filename='error_log.log', level=logging.ERROR, 
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger=logging.getLogger(__name__)

## Defining functions

In [ ]:
#selenium login
def login(input_driver):
        driver = input_driver
        driver.get("https://www.instagram.com/accounts/login")
        driver.implicitly_wait(5)
        try:
            driver.find_element_by_xpath("/html/body/div[2]/div/div/button[1]").click()
        except Exception as e:
            pass
        elem_cookie = driver.find_element_by_xpath('/html/body/div[4]/div/div/button[1]').click()
        time.sleep(1) 
        elem_user = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
        elem_user.send_keys(login_data[0])
        time.sleep(2)

        elem_pass = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
        elem_pass.send_keys(login_data[1])
        time.sleep(1)
        
    
        elem_login = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()

In [ ]:
driver = webdriver.Chrome(executable_path = "my path")
login(driver)

In [ ]:
# fetch the lists in the current dashboard
api = API(token)
for a_list in api.lists():
    print(a_list)

In [ ]:
today =  datetime.datetime.now()
today.isoformat()

In [ ]:
#API Post Abfrage
def post_report(list_id, API_token):
    api = API(API_token)
    collected_posts = []
    yesterday = today - datetime.timedelta(days = 1)
    y = str(today - datetime.timedelta(days = 1))
    for a_post in api.posts(listIds = list_id, 
                            count = -1,
                            batchSize = 100,
                            sortBy = 'date',
                            startDate = yesterday.isoformat(), 
                            endDate = today.isoformat(), 
                            timeframe = None,
                            ):
        parsed_post = {}
        parsed_post['id'] = a_post['id'] 
        parsed_post['platform_id'] = a_post['platformId']
        parsed_post['date'] = a_post['date']
        parsed_post['updated'] = a_post['updated']
        parsed_post['type'] = a_post['type']
        parsed_post['post_url'] = a_post['postUrl']
        parsed_post['subscriber_count'] = a_post['subscriberCount']
        parsed_post['language_code'] = a_post['languageCode']
        parsed_post['score'] = a_post['score']
        parsed_post['account_id'] = a_post['account']['id']
        parsed_post['name'] = a_post['account']['name']
        parsed_post['username'] = a_post['account']['handle']                                
        parsed_post['verified'] = a_post['account']['verified']
        parsed_post['actual_likes'] = a_post ['statistics']['actual']['favoriteCount']                               
        parsed_post['actual_comment_count'] = a_post['statistics']['actual']['commentCount']
        parsed_post['expected_likes'] = a_post['statistics']['expected']['favoriteCount']
        parsed_post['expected_comment_count']= a_post['statistics']['expected']['commentCount']                              
        collected_posts.append(parsed_post)
        time.sleep(0.5)
    df_ct = pd.DataFrame(collected_posts)
    df_ct.to_csv(r"{}/df_{}.tsv".format(my_path, str(yesterday)[:10]), encoding = "utf-8", index = False, sep ='\t') #tabs statt komma
    
    return df_ct

### historical Data

wird relevant, wenn wir nicht täglich Daten erheben wollen, sondern rückwirkend (zb. für das letze Jahr) Posts sammeln wollen


In [ ]:
#API Post Abfrage
def post_report_his(list_id, API_token):
    api = API(API_token)
    collected_posts = []
    today =  datetime.datetime.now() 

    for a_post in api.posts(listIds = list_id, 
                            count = -1,
                            batchSize = 100,
                            sortBy = 'date',
                            #startDate =  
                            endDate = today.isoformat(), 
                            timeframe = None,
                            ):
        # do something with the post
        parsed_post = {}
        parsed_post['id'] = a_post['id'] 
        parsed_post['date'] = a_post['date']
        parsed_post['updated'] = a_post['updated']
        parsed_post['type'] = a_post['type']
        parsed_post['post_url'] = a_post['postUrl']
        parsed_post['subscriber_count'] = a_post['subscriberCount']
        parsed_post['language_code'] = a_post['languageCode']
        parsed_post['score'] = a_post['score']
        parsed_post['account_id'] = a_post['account']['id']
        parsed_post['name'] = a_post['account']['name']
        parsed_post['username'] = a_post['account']['handle']                                
        parsed_post['verified'] = a_post['account']['verified']
        parsed_post['actual_likes'] = a_post ['statistics']['actual']['favoriteCount']                               
        parsed_post['actual_comment_count'] = a_post['statistics']['actual']['commentCount']
        parsed_post['expected_likes'] = a_post['statistics']['expected']['favoriteCount']
        parsed_post['expected_comment_count']= a_post['statistics']['expected']['commentCount']                              
        collected_posts.append(parsed_post)
        time.sleep(0.5)
    df_ct = pd.DataFrame(collected_posts)
    df_ct.to_csv("/data/history_df.tsv", encoding = "utf-8", index = False, sep ='\t') #tabs statt komma
    
    return df_ct

In [ ]:
#Email bei Fehlermeldung
def error_notficiation (text, text2):
    with open("Email_credentials.txt", "rt") as f: #obacht: funktioniert nur mit aktuellem Dateipfad
        login_data = f.read().split()
    
    s = smtplib.SMTP(host = 'mail.gmx.net', port = 587)
    s.starttls()
    s.login(login_data[0], login_data[1])
    
    error = text
    email = "my email"
    message = "Fehlermeldung: {} with {}".format(error, text2)

    msg = MIMEMultipart()       # create a message

    # setup the parameters of the message
    msg['From'] = "my email"
    msg['To'] = email
    msg['Subject'] = "Unexpected Error"

    # add in the message body
    msg.attach(MIMEText(message, 'plain'))

    # send the message va the server set up earlier.
    s.send_message(msg)

In [ ]:
#error counter
def error_counter(post_url):
    error_df = pd.read_csv(r"{}/error_df.tsv".format(my_path), encoding = "utf-8", sep ='\t')
    
    #if (! post_urlpost_url not in error_df["post_url"].tolist()():
    if post_url not in error_df["post_url"].tolist(): #neu
        error_df_new = error_df.append({'post_url':post_url,'attempt': 1,'report_date': str(today)}, ignore_index= True)
        error_df_new.to_csv(r"{}/data/error_df.tsv".format(my_path), 
                        encoding = "utf-8", 
                        sep = '\t', 
                        index = False)
    elif int(error_df.loc[error_df["post_url"] == post_url, "attempt"].item()) <= 3:
        error_df.loc[error_df["post_url"] == post_url, "attempt"] = \
        error_df.loc[error_df["post_url"] == post_url, "attempt"] + 1 
        error_df = error_df.to_csv(r"{}/error_df.tsv".format(my_path), 
                         encoding = "utf-8", 
                         sep = '\t', 
                         index = False)
    
    else:
        print("already tried 3 times")


In [ ]:
## get report

def report_df():
    today = datetime.datetime.now()
    print(str(today))
    yesterday = today - datetime.timedelta(days = 1)
    print(today)
    #selenium
    print("login to Instagram")
    global driver 
    driver = webdriver.Chrome(executable_path = "{}/chromedriver".format(my_path))
    time.sleep(2)
    login(driver)
    counter = 1
    #post_report
    print("Request Report")
    df_report = post_report( "my list", token)
    return df_report

## schedule

In [ ]:
import schedule

In [ ]:
#define scheduler
schedule1 = schedule.Scheduler()


#run scheduler
schedule1.every().day.at("09:00").do(report_df)


while True:
    schedule1.run_pending()
    time.sleep(1) 